In [2]:
import decimal
from decimal import Decimal
from collections import defaultdict
import psycopg2
import pandas as pd
import csv

In [1]:
import decimal
from decimal import Decimal
from collections import defaultdict
import psycopg2
import pandas as pd
import csv

# Database connection credentials
db_credentials = {
    'dbname': 'fixtransactions',
    'user': 'scalp',
    'password': 'QAtr@de442',
    'host': '10.7.8.59',
    'port': '5433'
}
def process_trades(date, conn):
    traders_data = defaultdict(lambda: defaultdict(int))
    with conn.cursor() as cursor:
        query = f"""SELECT tag1, tag35, tag39, tag150, tag38, tag32,tag167
                    FROM fixmsg
                    WHERE tag1 IS NOT null
                    AND tag35 IS NOT null
                    AND tag39 IS NOT null
                    AND tag150 IS NOT null
                    AND tag38 IS NOT null
                    AND tag167 IS NOT null
                    AND tag32 IS NOT null
                    AND tag52 >= TIMESTAMP '{date}'
                    AND tag52 < TIMESTAMP '{date}' + INTERVAL '1 DAY'
                    LIMIT 1000000;
                """
        cursor.execute(query)
        trades = cursor.fetchall()

    for trade in trades:
        account = trade[0]
        msg_type = trade[1]
        order_status = trade[2]
        exec_type = trade[3]
        order_qty = trade[4]
        last_shares = trade[5]
        instrument = trade[6]


    # Count the number of messages sent by each account
        if msg_type == '8' and order_status == '0':
            traders_data[account]['messages'] += 1
        else:
            traders_data[account]['messages'] += 0
        
        # Count the number of orders executed by each account
        if msg_type == '8' and order_status == '2':
            traders_data[account]['orders_executed'] += 1
        else:
            traders_data[account]['orders_executed'] += 0
        
        # Count the number of orders cancelled by each account
        if msg_type == '8' and order_status == '4':
            traders_data[account]['orders_canceled'] += 1
        else:
            traders_data[account]['orders_canceled'] += 0
        # Count the number of equity-specific shares submitted by each account
        if msg_type == '8' and instrument == 'CS' and order_status == '0':
            traders_data[account]['equity_shares_submitted'] += order_qty
        else:
            traders_data[account]['equity_shares_submitted'] += 0
        # Count the number of equity-specific shares executed by each account
        if msg_type == '8' and instrument == 'CS' and order_status == '2':
            traders_data[account]['equity_shares_executed'] += order_qty
        else:
            traders_data[account]['equity_shares_executed'] += 0
        if msg_type == '8' and instrument == 'CS' and order_status == '4':
            traders_data[account]['equity_shares_canceled'] += order_qty
                # Count the number of equity-specific shares submitted by each account
        else:
            traders_data[account]['equity_shares_canceled'] += 0

        if msg_type == '8' and instrument == 'OPT' and order_status == '0':
            traders_data[account]['option_shares_submitted'] += order_qty
        else:
            traders_data[account]['option_shares_submitted'] += 0
        # Count the number of equity-specific shares executed by each account
        if msg_type == '8' and instrument == 'OPT' and order_status == '2':
            traders_data[account]['option_shares_executed'] += order_qty
        else:
            traders_data[account]['option_shares_executed'] += 0
        if msg_type == '8' and instrument == 'OPT' and order_status == '4':
            traders_data[account]['option_shares_canceled'] += order_qty
        else:
            traders_data[account]['option_shares_canceled'] += 0
    
    

    total_orders = sum([data['orders_executed'] + data['orders_canceled'] for account, data in traders_data.items()])
    total_executed = sum([data['orders_executed'] for account, data in traders_data.items()])
    total_canceled = sum([data['orders_canceled'] for account, data in traders_data.items()])

    total_equities_submitted = sum([data['equity_shares_submitted'] for account, data in traders_data.items()])
    total_equities_executed = sum([data['equity_shares_executed'] for account, data in traders_data.items()])
    total_equities_canceled = sum([data['equity_shares_canceled'] for account, data in traders_data.items()])

    total_options_submitted = sum([data['option_shares_submitted'] for account, data in traders_data.items()])
    total_options_executed = sum([data['option_shares_executed'] for account, data in traders_data.items()])
    total_options_canceled = sum([data['option_shares_canceled'] for account, data in traders_data.items()])

    percentage_executed = (total_executed / total_orders) * 100
    percentage_canceled = (total_canceled / total_orders) * 100
    percentage_equities_canceled = (total_equities_canceled / total_equities_submitted) * 100
    percentage_equities_executed = (total_equities_executed / total_equities_submitted) * 100
    percentage_options_canceled = (total_options_canceled / total_options_submitted) * 100
    percentage_options_executed = (total_options_executed / total_options_submitted) * 100

    print(f"total_orders: {total_orders}, total_executed: {total_executed}, total_canceled: {total_canceled}")
    print(f"total_equities_submitted: {total_equities_submitted}, total_equities_executed: {total_equities_executed}, total_equities_canceled: {total_equities_canceled}")
    print(f"total_options_submitted: {total_options_submitted}, total_options_executed: {total_options_executed}, total_options_canceled: {total_options_canceled}")
    print(f"percentage_executed: {percentage_executed:.1f}:, percentage_equities_executed: {percentage_equities_executed:.1f}, percentage_options_executed: {percentage_options_executed:.1f}")
    print(f"percentage_canceled: {percentage_canceled:.1f}:, percentage_equities_canceled: {percentage_equities_canceled:.1f}, percentage_options_camceled: {percentage_options_canceled:.1f}")

        
    return traders_data

if __name__ == "__main__":
    date = '2023-07-13'
    conn = psycopg2.connect(**db_credentials)
    traders_data = process_trades(date, conn)
    conn.close()

    with open('execution_ratio.csv', mode='w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Account', 'Messages', 'Orders Executed', 'Orders Cancelled', 
                         'Equity Shares Submitted', 'Equity Shares Executed', 'Equity Shares Cancelled',
                         'Option Shares Submitted', 'Option Shares Executed', 'Option Shares Cancelled'])
        
        for account, data in traders_data.items():
            writer.writerow([account, data['messages'], data['orders_executed'], data['orders_canceled'],
                             data['equity_shares_submitted'], data['equity_shares_executed'], data['equity_shares_canceled'],
                             data['option_shares_submitted'], data['option_shares_executed'], data['option_shares_canceled']])

    for account, data in traders_data.items():
        print(
            account,
            data['messages'], 
            data["orders_executed"], 
            data["orders_cancelled"], 
            data['equity_shares_submitted'], 
            data['equity_shares_executed'], 
            data['equity_shares_canceled'],
            data['option_shares_submitted'], 
            data['option_shares_executed'], 
            data['option_shares_canceled']
        )    



NameError: name 'psycopg2' is not defined

In [1]:
import psycopg2
import pandas as pd
from datetime import datetime
import concurrent.futures

def fetch_total_positions(cursor, date):
        query = f"""
        SELECT     
            tag1,
            tag55,
            SUM(CASE WHEN tag54 = '1' THEN tag38 ELSE 0 END) -
            SUM(CASE WHEN tag54 = '2' THEN tag38 ELSE 0 END) AS total_position 
        FROM fixmsg
        WHERE (tag167 = 'OPT' or tag167 = 'MLEG')  
            AND tag52 >= TIMESTAMP '{date}'
            AND tag52 < TIMESTAMP '{date}' + INTERVAL '1 DAY'
        GROUP BY tag1, tag55
        HAVING sum(case when tag54 = '1' then tag38 else 0 end) -
            sum(case when tag54 = '2' then tag38 else 0 end) <> 0

        UNION

        SELECT     
            tag1,
            tag55,
            SUM(CASE WHEN tag54 = '1' THEN tag38 ELSE 0 END) -
            SUM(CASE WHEN tag54 = '2' THEN tag38 ELSE 0 END) AS total_position 
        FROM fixmsg
        WHERE tag167 != 'OPT'   
            AND tag52 >= TIMESTAMP '{date}'
            AND tag52 < TIMESTAMP '{date}' + INTERVAL '1 DAY'
        GROUP BY tag1, tag55
        HAVING sum(case when tag54 = '1' then tag38 else 0 end) -
            sum(case when tag54 = '2' then tag38 else 0 end) <> 0;
        """

        cursor.execute(query)
        results = cursor.fetchall()

        positions_df = pd.DataFrame(results, columns=['account', 'symbol', 'total_position'])

        return positions_df

connection = psycopg2.connect(host="10.7.8.59", database="fixtransactions", user="scalp", password="QAtr@de442", port='5433')
cursor = connection.cursor()
date = '2023-05-08'
positions = fetch_total_positions(cursor, date)

In [4]:
positions
grouped_positions = positions.groupby(['account', 'symbol'])['total_position'].sum().reset_index()

grouped_positions.to_csv("grouped_positions_df.csv")